In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
from warnings import filterwarnings
from sklearn.metrics import mean_squared_error as mse
filterwarnings("ignore")

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_excel (r'/kaggle/input/doc-stocks/stocks tcs.xlsx',
                   usecols=['Date', 'Close*'], parse_dates=['Date'],index_col='Date')
stock_price = df[::-1].squeeze()
plt.figure(figsize=(17,5))
plt.plot(stock_price)
plt.title("Closing Price Adjusted",fontsize=20)
plt.show()

In [ ]:
y_train = stock_price.iloc[:30]
y_test = stock_price.iloc[30:]

def plot_pred(pred,title):
    plt.figure(figsize=(17,5))
    plt.plot(y_train,label='Train')
    plt.plot(y_test,label='Actual')
    plt.plot(pred,label='Predicted')
    plt.ylabel("Stock prices")
    plt.title(title,fontsize=20)
    plt.legend()
    plt.show()

In [ ]:
print("start:",y_test.index.min())
print("end:",y_test.index.max())

In [ ]:
y_test.shape

# Auto Regression
The prediction will be a linear combination of the past values. 

In [ ]:
from statsmodels.tsa.ar_model import AR
ar_model = AR(y_train).fit()
y_ar = ar_model.predict(30,39)
y_ar = y_ar.reset_index(drop=True)
y_ar.index = y_test.index
mse(y_ar,y_test)

In [ ]:
plot_pred(y_ar,"Autoregression")

# Moving Average
  Moving Average is not the predictions based on the moving average of the previous values, instead it is the moving average of the residuals of the previous values.


In [ ]:
from statsmodels.tsa.arima_model import ARMA
ma = ARMA(y_train, order=(0, 1)).fit()
y_mam = ma.predict(30,39)
y_mam.index = y_test.index
mse(y_mam,y_test)


In [ ]:
plot_pred(y_mam,"Moving Average(MA)")

# ARMA
This is a combination of Auto Regression and Moving Average.

In [ ]:
arma = ARMA(y_train,order=(1,1)).fit()
y_arma =arma.predict(30,39)
y_arma.index = y_test.index
mse(y_arma,y_test)


In [ ]:
plot_pred(y_arma,"ARMA Model")

# ARIMA
But this unsuitable for Series with Seasonality. We have a seasonality in our data, but we do not have a trend. Lets construct an ARIMA model for this.

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
arima = ARIMA(y_train,order=(2,0,2)).fit(disp=False)
y_arima = arima.predict(30,39)
y_arima.index = y_test.index
mse(y_arima,y_test)

In [ ]:
plot_pred(y_arima,"ARIMA Model")

We donot have a good prediction as there is no significant trend in the data!

Ref: https://machinelearningmastery.com/time-series-forecasting-methods-in-python-cheat-sheet/